In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv("./output.csv")
df

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,WikiId,Name_wiki,Age,Hometown,Embarked_wiki,Destination,Lifeboat,Body,Class,Family,Embarked_place_uncertain,Country,Region,Body_Found,Survived
0,1,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,Unknown,S,691.0,"Braund, Mr. Owen Harris",22.0,"Bridgerule, Devon, England",S,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0,1,False,UK,England,False,0.0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C,90.0,"Cumings, Mrs. Florence Briggs (née Thayer)",35.0,"New York, New York, US",C,"New York, New York, US",4,NaN,1.0,1,False,US,US,False,1.0
2,3,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,Unknown,S,865.0,"Heikkinen, Miss Laina",26.0,"Jyväskylä, Finland",S,New York City,14?,NaN,3.0,0,False,Russian Empire,Finland,False,1.0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S,127.0,"Futrelle, Mrs. Lily May (née Peel)",35.0,"Scituate, Massachusetts, US",S,"Scituate, Massachusetts, US",D,NaN,1.0,1,False,US,US,False,1.0
4,5,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,Unknown,S,627.0,"Allen, Mr. William Henry",35.0,"Birmingham, West Midlands, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,"Spector, Mr. Woolf",male,0,0,A.5. 3236,8.0500,Unknown,S,1227.0,"Spector, Mr. Woolf",23.0,"London, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0
1305,1306,1,"Oliva y Ocana, Dona. Fermina",female,0,0,PC 17758,108.9000,C105,C,229.0,"and maid, Doña Fermina Oliva y Ocana",39.0,"Madrid, Spain",C,"New York, New York, US",8,NaN,1.0,0,False,Spain,Spain,False,1.0
1306,1307,3,"Saether, Mr. Simon Sivertsen",male,0,0,SOTON/O.Q. 3101262,7.2500,Unknown,S,1169.0,"Sæther, Mr. Simon Sivertsen",43.0,"Skaun, Sør-Trøndelag, Norway",S,US,NaN,32MB,3.0,0,False,Norway,Norway,True,0.0
1307,1308,3,"Ware, Mr. Frederick",male,0,0,359309,8.0500,Unknown,S,1289.0,"Ware, Mr. Frederick William",34.0,"Greenwich, London, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0


In [2]:
def extract_title(name):
    title = name.split(',')[1].split('.')[0].strip()
    return title
df['Title'] = df['Name'].apply(extract_title)
print(df[['Name', 'Title']].head())
title_counts = df['Title'].value_counts()
print(title_counts)

                                                Name Title
0                            Braund, Mr. Owen Harris    Mr
1  Cumings, Mrs. John Bradley (Florence Briggs Th...   Mrs
2                             Heikkinen, Miss. Laina  Miss
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)   Mrs
4                           Allen, Mr. William Henry    Mr
Title
Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: count, dtype: int64


In [3]:
replace_title = {
    "Mlle": "Miss",
    "Mme": "Mrs",
    "Dona": "Lady",
}
df['Title'] = df['Title'].replace(replace_title)
df['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Ms', 'Major',
       'Lady', 'Sir', 'Col', 'Capt', 'the Countess', 'Jonkheer'],
      dtype=object)

In [4]:
unknown_count = df['Cabin'].eq("Unknown").sum()
print(unknown_count)

1014


In [5]:
import re

def split_ticket(ticket):
    letters = ''.join(re.findall(r'[A-Za-z]', ticket))
    numbers = ''.join(re.findall(r'\d+', ticket))
    return pd.Series([letters, numbers])

df[['Cabin_Letters', 'Cabin_Numbers']] = df['Cabin'].apply(split_ticket)


In [6]:
ticket_group = df.groupby(['Ticket', 'Cabin'])['Fare'].agg(['count', 'mean']).reset_index()
ticket_group .columns = ['Ticket', 'Cabin', 'Passenger Count', 'Fare']
ticket_group[['Cabin_Letters', 'Cabin_Numbers']] = ticket_group['Cabin'].apply(split_ticket)
ticket_group[['Ticket_Letters', 'Ticket_Numbers']] = ticket_group['Ticket'].str.split(n=1, expand=True)
mask = ticket_group['Ticket_Numbers'].isnull()  # 创建掩码
ticket_group.loc[mask, ['Ticket_Letters', 'Ticket_Numbers']] = ticket_group.loc[mask, ['Ticket_Numbers', 'Ticket_Letters']].values
ticket_group['Ticket_Letters'] = ticket_group['Ticket_Letters'].fillna('')  # 将 NaN 替换为空字符串
ticket_group_nodu = ticket_group.drop_duplicates(subset='Ticket', keep=False)
ticket_group_nodu = ticket_group_nodu.assign(Real_Fare=ticket_group_nodu['Fare'] / ticket_group_nodu['Passenger Count'])
ticket_group_nodu[ticket_group_nodu['Cabin_Letters'] == 'F']

,Ticket,Cabin,Passenger Count,Fare,Cabin_Letters,Cabin_Numbers,Ticket_Letters,Ticket_Numbers,Real_Fare
165,230080,F2,3,26.00,F,2,,230080,8.666667
166,230136,F4,4,39.00,F,4,,230136,9.750000
222,244368,F2,1,13.00,F,2,,244368,13.000000
231,248733,F33,1,13.00,F,33,,248733,13.000000
232,248734,F,1,13.00,F,,,248734,13.000000
686,383121,F38,1,7.75,F,38,,383121,7.750000
780,C.A. 29395,F33,1,10.50,F,33,C.A.,29395,10.500000
791,C.A. 34260,F33,1,10.50,F,33,C.A.,34260,10.500000
967,W./C. 14266,F33,1,10.50,F,33,W./C.,14266,10.500000


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def train_and_predict(data, feature, label):
    
    # Encode the categorical feature using one-hot encoding

    X = data[feature]
    y = data[label]    

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    X_train, X_test = X, X
    y_train, y_test = y_encoded, y_encoded

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    # Ensure predictions are transformed back to original labels
    predictions_labels = le.inverse_transform(predictions)

    report = classification_report(
        y_test, 
        predictions, 
        target_names=le.classes_,  # Use all classes
        zero_division=0  # Handle cases where a class may not be present in predictions
    )
    
    print("分类报告:\n", report)

    return model, le

def predict(data, feature, model, le):
    X = data[feature]
    predictions = model.predict(X)
    predictions_labels = le.inverse_transform(predictions)
    return predictions_labels
    

# Filter the data to remove 'Unknown' Cabin_Letters
to_check = ["C", "D", "B", "E", "A", "T", "F", "G"]
data = ticket_group_nodu[ticket_group_nodu['Cabin_Letters'].isin(to_check)]

# Call the function with the modified feature list
label_encoder = LabelEncoder()
data['Ticket_Letters'] = label_encoder.fit_transform(data['Ticket_Letters'])
model, le = train_and_predict(data, ['Ticket_Letters', 'Real_Fare'], 'Cabin_Letters')

分类报告:
               precision    recall  f1-score   support

           A       0.77      0.56      0.65        18
           B       0.78      0.70      0.74        20
           C       0.61      0.91      0.73        34
           D       0.86      0.73      0.79        26
           E       1.00      0.62      0.76        21
           F       0.75      1.00      0.86         9
           G       1.00      1.00      1.00         2
           T       0.00      0.00      0.00         1

    accuracy                           0.75       131
   macro avg       0.72      0.69      0.69       131
weighted avg       0.78      0.75      0.74       131



C:\Users\90586\AppData\Local\Temp\ipykernel_35044\70972700.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Ticket_Letters'] = label_encoder.fit_transform(data['Ticket_Letters'])


In [8]:
df[['Cabin_Letters', 'Cabin_Numbers']] = df['Cabin'].apply(split_ticket)
df[['Ticket_Letters', 'Ticket_Numbers']] = df['Ticket'].str.split(n=1, expand=True)
mask = df['Ticket_Numbers'].isnull()  # 创建掩码
df.loc[mask, ['Ticket_Letters', 'Ticket_Numbers']] = df.loc[mask, ['Ticket_Numbers', 'Ticket_Letters']].values
df['Ticket_Letters'] = df['Ticket_Letters'].fillna('')  # 将 NaN 替换为空字符串
df

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,WikiId,Name_wiki,Age,Hometown,Embarked_wiki,Destination,Lifeboat,Body,Class,Family,Embarked_place_uncertain,Country,Region,Body_Found,Survived,Title,Cabin_Letters,Cabin_Numbers,Ticket_Letters,Ticket_Numbers
0,1,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,Unknown,S,691.0,"Braund, Mr. Owen Harris",22.0,"Bridgerule, Devon, England",S,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0,1,False,UK,England,False,0.0,Mr,Unknown,,A/5,21171
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C,90.0,"Cumings, Mrs. Florence Briggs (née Thayer)",35.0,"New York, New York, US",C,"New York, New York, US",4,NaN,1.0,1,False,US,US,False,1.0,Mrs,C,85,PC,17599
2,3,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,Unknown,S,865.0,"Heikkinen, Miss Laina",26.0,"Jyväskylä, Finland",S,New York City,14?,NaN,3.0,0,False,Russian Empire,Finland,False,1.0,Miss,Unknown,,STON/O2.,3101282
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S,127.0,"Futrelle, Mrs. Lily May (née Peel)",35.0,"Scituate, Massachusetts, US",S,"Scituate, Massachusetts, US",D,NaN,1.0,1,False,US,US,False,1.0,Mrs,C,123,,113803
4,5,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,Unknown,S,627.0,"Allen, Mr. William Henry",35.0,"Birmingham, West Midlands, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,,373450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,"Spector, Mr. Woolf",male,0,0,A.5. 3236,8.0500,Unknown,S,1227.0,"Spector, Mr. Woolf",23.0,"London, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,A.5.,3236
1305,1306,1,"Oliva y Ocana, Dona. Fermina",female,0,0,PC 17758,108.9000,C105,C,229.0,"and maid, Doña Fermina Oliva y Ocana",39.0,"Madrid, Spain",C,"New York, New York, US",8,NaN,1.0,0,False,Spain,Spain,False,1.0,Lady,C,105,PC,17758
1306,1307,3,"Saether, Mr. Simon Sivertsen",male,0,0,SOTON/O.Q. 3101262,7.2500,Unknown,S,1169.0,"Sæther, Mr. Simon Sivertsen",43.0,"Skaun, Sør-Trøndelag, Norway",S,US,NaN,32MB,3.0,0,False,Norway,Norway,True,0.0,Mr,Unknown,,SOTON/O.Q.,3101262
1307,1308,3,"Ware, Mr. Frederick",male,0,0,359309,8.0500,Unknown,S,1289.0,"Ware, Mr. Frederick William",34.0,"Greenwich, London, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,,359309


In [9]:
merged_df = pd.merge(df, ticket_group_nodu, on='Ticket', how='left')
merged_df

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare_x,Cabin_x,Embarked,WikiId,Name_wiki,Age,Hometown,Embarked_wiki,Destination,Lifeboat,Body,Class,Family,Embarked_place_uncertain,Country,Region,Body_Found,Survived,Title,Cabin_Letters_x,Cabin_Numbers_x,Ticket_Letters_x,Ticket_Numbers_x,Cabin_y,Passenger Count,Fare_y,Cabin_Letters_y,Cabin_Numbers_y,Ticket_Letters_y,Ticket_Numbers_y,Real_Fare
0,1,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,Unknown,S,691.0,"Braund, Mr. Owen Harris",22.0,"Bridgerule, Devon, England",S,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0,1,False,UK,England,False,0.0,Mr,Unknown,,A/5,21171,Unknown,1.0,7.2500,Unknown,,A/5,21171,7.25000
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C,90.0,"Cumings, Mrs. Florence Briggs (née Thayer)",35.0,"New York, New York, US",C,"New York, New York, US",4,NaN,1.0,1,False,US,US,False,1.0,Mrs,C,85,PC,17599,C85,2.0,71.2833,C,85,PC,17599,35.64165
2,3,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,Unknown,S,865.0,"Heikkinen, Miss Laina",26.0,"Jyväskylä, Finland",S,New York City,14?,NaN,3.0,0,False,Russian Empire,Finland,False,1.0,Miss,Unknown,,STON/O2.,3101282,Unknown,1.0,7.9250,Unknown,,STON/O2.,3101282,7.92500
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S,127.0,"Futrelle, Mrs. Lily May (née Peel)",35.0,"Scituate, Massachusetts, US",S,"Scituate, Massachusetts, US",D,NaN,1.0,1,False,US,US,False,1.0,Mrs,C,123,,113803,C123,2.0,53.1000,C,123,,113803,26.55000
4,5,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,Unknown,S,627.0,"Allen, Mr. William Henry",35.0,"Birmingham, West Midlands, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,,373450,Unknown,1.0,8.0500,Unknown,,,373450,8.05000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,"Spector, Mr. Woolf",male,0,0,A.5. 3236,8.0500,Unknown,S,1227.0,"Spector, Mr. Woolf",23.0,"London, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,A.5.,3236,Unknown,1.0,8.0500,Unknown,,A.5.,3236,8.05000
1305,1306,1,"Oliva y Ocana, Dona. Fermina",female,0,0,PC 17758,108.9000,C105,C,229.0,"and maid, Doña Fermina Oliva y Ocana",39.0,"Madrid, Spain",C,"New York, New York, US",8,NaN,1.0,0,False,Spain,Spain,False,1.0,Lady,C,105,PC,17758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1306,1307,3,"Saether, Mr. Simon Sivertsen",male,0,0,SOTON/O.Q. 3101262,7.2500,Unknown,S,1169.0,"Sæther, Mr. Simon Sivertsen",43.0,"Skaun, Sør-Trøndelag, Norway",S,US,NaN,32MB,3.0,0,False,Norway,Norway,True,0.0,Mr,Unknown,,SOTON/O.Q.,3101262,Unknown,1.0,7.2500,Unknown,,SOTON/O.Q.,3101262,7.25000
1307,1308,3,"Ware, Mr. Frederick",male,0,0,359309,8.0500,Unknown,S,1289.0,"Ware, Mr. Frederick William",34.0,"Greenwich, London, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,,359309,Unknown,1.0,8.0500,Unknown,,,359309,8.05000


In [10]:
df['Real_Fare'] = pd.NA
for ticket in merged_df['Ticket']:
    if ticket in df['Ticket'].values:  # Use .values for better performance
        df.loc[df['Ticket'] == ticket, 'Real_Fare'] = merged_df.loc[merged_df['Ticket'] == ticket, 'Real_Fare'].values[0]
df


,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,WikiId,Name_wiki,Age,Hometown,Embarked_wiki,Destination,Lifeboat,Body,Class,Family,Embarked_place_uncertain,Country,Region,Body_Found,Survived,Title,Cabin_Letters,Cabin_Numbers,Ticket_Letters,Ticket_Numbers,Real_Fare
0,1,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,Unknown,S,691.0,"Braund, Mr. Owen Harris",22.0,"Bridgerule, Devon, England",S,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0,1,False,UK,England,False,0.0,Mr,Unknown,,A/5,21171,7.25
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C,90.0,"Cumings, Mrs. Florence Briggs (née Thayer)",35.0,"New York, New York, US",C,"New York, New York, US",4,NaN,1.0,1,False,US,US,False,1.0,Mrs,C,85,PC,17599,35.64165
2,3,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,Unknown,S,865.0,"Heikkinen, Miss Laina",26.0,"Jyväskylä, Finland",S,New York City,14?,NaN,3.0,0,False,Russian Empire,Finland,False,1.0,Miss,Unknown,,STON/O2.,3101282,7.925
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S,127.0,"Futrelle, Mrs. Lily May (née Peel)",35.0,"Scituate, Massachusetts, US",S,"Scituate, Massachusetts, US",D,NaN,1.0,1,False,US,US,False,1.0,Mrs,C,123,,113803,26.55
4,5,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,Unknown,S,627.0,"Allen, Mr. William Henry",35.0,"Birmingham, West Midlands, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,,373450,8.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,"Spector, Mr. Woolf",male,0,0,A.5. 3236,8.0500,Unknown,S,1227.0,"Spector, Mr. Woolf",23.0,"London, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,A.5.,3236,8.05
1305,1306,1,"Oliva y Ocana, Dona. Fermina",female,0,0,PC 17758,108.9000,C105,C,229.0,"and maid, Doña Fermina Oliva y Ocana",39.0,"Madrid, Spain",C,"New York, New York, US",8,NaN,1.0,0,False,Spain,Spain,False,1.0,Lady,C,105,PC,17758,NaN
1306,1307,3,"Saether, Mr. Simon Sivertsen",male,0,0,SOTON/O.Q. 3101262,7.2500,Unknown,S,1169.0,"Sæther, Mr. Simon Sivertsen",43.0,"Skaun, Sør-Trøndelag, Norway",S,US,NaN,32MB,3.0,0,False,Norway,Norway,True,0.0,Mr,Unknown,,SOTON/O.Q.,3101262,7.25
1307,1308,3,"Ware, Mr. Frederick",male,0,0,359309,8.0500,Unknown,S,1289.0,"Ware, Mr. Frederick William",34.0,"Greenwich, London, England",S,New York City,NaN,NaN,3.0,0,False,UK,England,False,0.0,Mr,Unknown,,,359309,8.05


In [11]:
tmp_df = df[['Real_Fare', 'Ticket_Letters']].copy()
tmp_df['Ticket_Letters'] = label_encoder.fit_transform(tmp_df['Ticket_Letters'])
df['Predict_Cabin'] = predict(tmp_df, ['Ticket_Letters', 'Real_Fare'], model, le)

In [12]:
import numpy as np
df.loc[df['Real_Fare'].isna(), 'Predict_Cabin'] = 'Unknown'
df.loc[df['Real_Fare'].isna(), 'Predict_Cabin'].unique()

array(['Unknown'], dtype=object)

In [13]:
df.loc[df['Predict_Cabin'] != 'Unknown', 'Predict_Cabin'].count()

1200

In [14]:

df['Predict_Cabin'] = np.where((df['Cabin_Letters']!= 'Unknown') & (df['Predict_Cabin'] != df['Cabin_Letters']), df['Cabin_Letters'], df['Predict_Cabin'])

In [15]:
df['Predict_Cabin'].unique()

array(['D', 'C', 'E', 'F', 'G', 'A', 'CCC', 'B', 'FG', 'DD', 'BB', 'FE',
       'Unknown', 'CC', 'BBBB', 'T', 'BBB', 'EE'], dtype=object)

In [16]:
df.loc[df['Predict_Cabin'] != 'Unknown', 'Predict_Cabin'].count()

1293

In [17]:
df.to_csv('extend.csv')